In [1]:
%load_ext autoreload
%autoreload now

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "MIG-08137aa2-e69b-5e74-8390-7997329b1336"
# os.environ["WORLD_SIZE"] = "1"

# Download and convert data

In [3]:
from tqdm import tqdm

from document_segmentation.pagexml.annotations.renate_analysis import RenateAnalysis
from document_segmentation.settings import RENATE_ANALYSIS_DIR

N = None

RENATE_ANALYSIS_DIR.mkdir(parents=True, exist_ok=True)

sheet = RenateAnalysis()


existing_docs = {
    path.stem for path in RENATE_ANALYSIS_DIR.glob("Globdoc_*.json") if path.is_file()
}

for document in tqdm(
    sheet.to_documents(n=N, skip_ids=existing_docs),
    total=(N or len(sheet)) - len(existing_docs),
    desc="Writing documents",
    unit="doc",
):
    document_file = RENATE_ANALYSIS_DIR / f"{document.id}.json"

    with document_file.open("xt") as f:
        f.write(document.model_dump_json())
        f.write("\n")

Writing documents: 0doc [00:00, ?doc/s]


In [4]:
import logging

from tqdm import tqdm

from document_segmentation.pagexml.annotations.renate_analysis import RenateAnalysisInv
from document_segmentation.settings import RENATE_ANALYSIS_DIR, RENATE_ANALYSIS_SHEETS

N = None


sheet = RenateAnalysisInv(RENATE_ANALYSIS_SHEETS[0])  # TODO: use both sheets

for document in tqdm(
    sheet.to_documents(n=N), desc="Writing documents", unit="doc", total=26
):
    document_file = RENATE_ANALYSIS_DIR / f"{document.id}.json"

    if document_file.exists():
        logging.info(f"Document {document.id} already exists, skipping")
    else:
        with document_file.open("xt") as f:
            f.write(document.model_dump_json())
            f.write("\n")

Writing documents:   0%|          | 0/26 [00:00<?, ?doc/s]

Writing documents: 100%|██████████| 26/26 [00:10<00:00,  2.55doc/s]


# Load Data

In [18]:
%autoreload now

In [19]:
TRAINING_DATA = 0.8

In [20]:
from document_segmentation.model.dataset import DocumentDataset

dataset: DocumentDataset = DocumentDataset.from_dir(RENATE_ANALYSIS_DIR)
len(dataset)

Reading JSON files: 100%|██████████| 104/104 [00:00<00:00, 176.63file/s]


2184

In [21]:
dataset._class_counts()

Counter({<Label.IN: 1>: 1907,
         <Label.BEGIN: 0>: 104,
         <Label.END: 2>: 100,
         <Label.OUT: 3>: 73})

In [22]:
dataset.class_weights()

[20.8, 1.1446540880503144, 21.623762376237625, 29.513513513513512]

In [24]:
training_data, test_data = dataset.split(TRAINING_DATA)

In [26]:
training_data._class_counts()

Counter({<Label.IN: 1>: 1591,
         <Label.BEGIN: 0>: 83,
         <Label.END: 2>: 81,
         <Label.OUT: 3>: 67})

In [27]:
test_data._class_counts()

Counter({<Label.IN: 1>: 316,
         <Label.BEGIN: 0>: 21,
         <Label.END: 2>: 19,
         <Label.OUT: 3>: 6})

# Train Model

In [28]:
import torch

BATCH_SIZE = 32
EPOCHS = 10
WEIGHTS = torch.Tensor(dataset.class_weights())  # For an imbalanced dataset

In [ ]:
%autoreload now

In [29]:
from document_segmentation.model.page_sequence_tagger import PageSequenceTagger

tagger = PageSequenceTagger()

In [30]:
tagger._device

'mps'

In [31]:
tagger

PageSequenceTagger(
  (_page_embedding): PageEmbedding(
    (_region_model): RegionEmbeddingSentenceTransformer(
      (_transformer_model): SentenceTransformer(
        (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: RobertaModel 
        (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
      )
      (_region_type): Embedding(9, 16)
      (_linear): Linear(in_features=784, out_features=512, bias=True)
    )
    (_rnn): LSTM(512, 256, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
    (_linear): Linear(in_features=512, out_features=256, bias=True)
  )
  (_rnn): LSTM(256, 256, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
  (_linear): Linear(in_features=512, out_features=4, bias=True)
  (_soft

In [32]:
tagger.train_(training_data, EPOCHS, BATCH_SIZE, WEIGHTS.to(tagger._device))


/Users/carstenschnober/LAHTeR/workspace/document-segmentation/.venv/lib/python3.11/site-packages/tqdm/std.py:636: TqdmWarning: clamping frac to range [0, 1]
116batch [01:44,  1.11batch/s]
Reading JSON files:   0%|          | 0/104 [09:17<?, ?file/s]

Current allocated memory (MPS): 1182 MB
Driver allocated memory (MPS): 2906 MB
[Loss:	6.392]


116batch [00:06, 18.18batch/s]
Reading JSON files:   0%|          | 0/104 [09:23<?, ?file/s]

Current allocated memory (MPS): 1182 MB
Driver allocated memory (MPS): 2860 MB
[Loss:	4.474]


116batch [00:06, 18.00batch/s]
Reading JSON files:   0%|          | 0/104 [09:30<?, ?file/s]

...

Current allocated memory (MPS): 1182 MB
Driver allocated memory (MPS): 2842 MB
[Loss:	3.569]


116batch [00:05, 19.60batch/s]
Reading JSON files:   0%|          | 0/104 [10:05<?, ?file/s]

Current allocated memory (MPS): 1182 MB
Driver allocated memory (MPS): 2842 MB
[Loss:	3.568]


116batch [00:05, 19.56batch/s]
Reading JSON files:   0%|          | 0/104 [10:11<?, ?file/s]

Current allocated memory (MPS): 1182 MB
Driver allocated memory (MPS): 2842 MB
[Loss:	3.567]


# Evaluate Model

In [33]:
import csv
import sys

from torcheval.metrics import (
    MulticlassAccuracy,
    MulticlassF1Score,
    MulticlassPrecision,
    MulticlassRecall,
)
from tqdm import tqdm

from document_segmentation.pagexml.datamodel.label import Label

writer = csv.DictWriter(
    sys.stdout,
    fieldnames=("Predicted", "Actual", "Page ID", "Text", "Scores"),
    delimiter="\t",
)

writer.writeheader()

accuracy = MulticlassAccuracy(num_classes=len(Label))
precision = MulticlassPrecision(average=None, num_classes=len(Label))
recall = MulticlassRecall(average=None, num_classes=len(Label))
f1_score = MulticlassF1Score(average=None, num_classes=len(Label))

for batch in tqdm(
    test_data.batches(BATCH_SIZE), total=len(test_data) / BATCH_SIZE, unit="batch"
):
    predicted = tagger(batch)
    labels = batch.labels()

    _labels = torch.Tensor([label.value for label in labels]).to(int)
    accuracy.update(predicted, _labels)
    precision.update(predicted, _labels)
    recall.update(predicted, _labels)
    f1_score.update(predicted, _labels)

    for page, pred, label in zip(batch.pages, predicted, labels):
        pred_label = Label(pred.argmax().item())
        # if pred_label != Label.IN or label != Label.IN:
        writer.writerow(
            {
                "Predicted": pred_label.name,
                "Actual": label.name,
                "Page ID": page.doc_id,
                "Text": page.text(delimiter="; ")[:50],
                "Scores": str(pred.tolist()),
            }
        )

Reading JSON files:   0%|          | 0/104 [10:23<?, ?file/s]


Predicted	Actual	Page ID	Text	Scores


 18%|█▊        | 2/11.3125 [00:00<00:03,  2.72batch/s]

BEGIN	BEGIN	NL-HaNA_1.04.02_1060_0435.jpg	Alsoo het schip der Goes, als t'Jacht Cleijn Enckh	[0.9996637105941772, 0.0002524361771065742, 5.55374936084263e-05, 2.8267319066799246e-05]
IN	IN	NL-HaNA_1.04.02_1060_0436.jpg	Op heden den 5 feb. @ 1614, door beroep vanden E. 	[0.0012747891014441848, 0.997553288936615, 0.0011668851366266608, 5.150438937562285e-06]
IN	IN	NL-HaNA_1.04.02_1060_0437.jpg	@ 1615. Dondergeschreven te soonen; Soo Is bij d' 	[0.0007243358995765448, 0.9982724189758301, 0.0009999239118769765, 3.3418723432987463e-06]
IN	IN	NL-HaNA_1.04.02_1060_0438.jpg	Den 19 feb. . vernomen hebbende op de factorije to	[0.0006676050252281129, 0.9983349442481995, 0.0009944145567715168, 3.1268480142898625e-06]
IN	IN	NL-HaNA_1.04.02_1060_0439.jpg	Ert Dircx wttgevaren voor Soldaet, opt schip Banta	[0.0006568318349309266, 0.9983606934547424, 0.000979323056526482, 3.1010447401058627e-06]
IN	IN	NL-HaNA_1.04.02_1060_0440.jpg	Comptoir deeart sullen gaen, omme door den opperco	[0.000655361975077539

 27%|██▋       | 3/11.3125 [00:01<00:03,  2.38batch/s]

...

OUT	OUT	NL-HaNA_1.04.02_1547_0360.jpg		[4.6434310206677765e-05, 2.5492503482382745e-05, 1.813627386582084e-05, 0.9999098777770996]
OUT	OUT	NL-HaNA_1.04.02_1547_0361.jpg		[1.5475776308448985e-05, 8.291252925118897e-06, 6.33479203315801e-06, 0.9999698400497437]
OUT	OUT	NL-HaNA_1.04.02_1547_0362.jpg		[2.7481732104206458e-05, 1.5985984646249563e-05, 1.1455773346824571e-05, 0.9999450445175171]
BEGIN	BEGIN	NL-HaNA_1.04.02_1547_0363.jpg	Jnstructie voor den onder„; Sonsbeek, vertreckende	[0.9995558857917786, 0.00018646824173629284, 3.85285857191775e-05, 0.00021914199169259518]
IN	IN	NL-HaNA_1.04.02_1547_0364.jpg	op te wakkeren sijn, soo meede te Ervaaren wat; Ef	[0.002504652366042137, 0.9966664910316467, 0.0007734567625448108, 5.537641845876351e-05]
IN	IN	NL-HaNA_1.04.02_1547_0365.jpg	Van ons gering vermoogen, voort te setten onder an	[0.0008247647783719003, 0.9984316229820251, 0.0007315014954656363, 1.215212614624761e-05]
IN	IN	NL-HaNA_1.04.02_1547_0366.jpg	aanslaagen der picaten en roovers, 

23batch [00:29,  1.13s/batch]

BEGIN	BEGIN	NL-HaNA_1.04.02_1083_0013.jpg	adn. 6=en augustij Anoo 1624; Octe der kerckelijck	[0.999489426612854, 0.00037690700264647603, 8.722964412299916e-05, 4.6380584535654634e-05]
IN	IN	NL-HaNA_1.04.02_1083_0014.jpg		[0.006468998268246651, 0.9929623007774353, 0.0003682940441649407, 0.0002003989793593064]
IN	IN	NL-HaNA_1.04.02_1083_0015.jpg	Int Jaer des Heeren onses Salichmaeckers Jesu Chri	[0.002301795408129692, 0.996840238571167, 0.0008491344633512199, 8.752758731134236e-06]
IN	IN	NL-HaNA_1.04.02_1083_0016.jpg	Kercken dienaers soo te lande als opde; Rhede alhi	[0.0005745316157117486, 0.9986787438392639, 0.0007434916333295405, 3.1757979286339832e-06]
IN	IN	NL-HaNA_1.04.02_1083_0017.jpg	Verlost moeten werden midtsgaders offf oock eenige	[0.0005185448681004345, 0.9987021684646606, 0.0007762503810226917, 2.94998312710959e-06]
IN	IN	NL-HaNA_1.04.02_1083_0018.jpg	Alsoo de Generale Comp. e. dageluckx boven haer ve	[0.0005135959945619106, 0.9986936450004578, 0.0007897705072537065, 2.92677

24batch [00:30,  1.29s/batch]

BEGIN	BEGIN	NL-HaNA_1.04.02_8696_0051.jpg	Van Siam onder dato 20: april 1737; Dag-register, 	[0.999670147895813, 0.0002444376586936414, 5.7068929891102016e-05, 2.8394919354468584e-05]
IN	IN	NL-HaNA_1.04.02_8696_0052.jpg	Van Siam onder dato 20:' april 1731; moesten verbl	[0.0013370545348152518, 0.9975607395172119, 0.0010970152216032147, 5.258822511677863e-06]
IN	IN	NL-HaNA_1.04.02_8696_0053.jpg	Van Siam onder dato 20: april 1737; zijn aen gesig	[0.0007060010102577507, 0.9982632994651794, 0.0010274903615936637, 3.2366833693231456e-06]
IN	IN	NL-HaNA_1.04.02_8696_0054.jpg	Siam onder dato 2 april 1737; den 8:e _=o alvroeg 	[0.0006481486489064991, 0.9983488321304321, 0.0010000128531828523, 3.0164023883116897e-06]
IN	IN	NL-HaNA_1.04.02_8696_0055.jpg	Van Siam onder dato: 20: april 1737; ons aengewees	[0.0006393356015905738, 0.9983567595481873, 0.0010009024990722537, 3.0001624509168323e-06]
IN	IN	NL-HaNA_1.04.02_8696_0056.jpg	Van Siam onder dato: 20: april 1737; Maart; die ge	[0.000636874872725

In [34]:
writer = csv.DictWriter(
    sys.stdout,
    fieldnames=["Metric"] + [label.name for label in Label],
    delimiter="\t",
)
writer.writeheader()

for metric in (precision, recall, f1_score):
    scores = {
        label.name: f"{score:.4f}"
        for label, score in zip(Label, metric.compute().tolist())
    }
    writer.writerow({"Metric": metric.__class__.__name__} | scores)

print(f"Accuracy ({accuracy.average} average):\t{accuracy.compute().item():.4f}")

Metric	BEGIN	IN	END	OUT
MulticlassPrecision	0.8400	0.9968	0.8571	0.7500
MulticlassRecall	1.0000	0.9715	0.9474	1.0000
MulticlassF1Score	0.9130	0.9840	0.9000	0.8571
Accuracy (micro average):	0.9724
